In [1]:
import findspark

findspark.init()
findspark.find()

from pyspark.sql import SparkSession


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
[2023-09-30 18:20:08] {NativeCodeLoader} WARN: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# spark.stop()

In [2]:
from os import getenv
import dotenv

dotenv.find_dotenv()
dotenv.load_dotenv()

False

In [4]:
spark = (
    SparkSession.builder.master("spark://spark-master:7077")
        .appName('test-app')
        .config(
            map={
                "spark.hadoop.fs.s3a.access.key": getenv("S3_ACCESS_KEY_ID"),
                "spark.hadoop.fs.s3a.secret.key": getenv("S3_SECRET_ACCESS_KEY"),
                "spark.hadoop.fs.s3a.endpoint": getenv("S3_ENDPOINT_URL"),
                "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
            }
        )
        .getOrCreate()
    )

In [5]:
df = spark.read.parquet('s3a://data-ice-lake-05/dev/source/transaction-service/object_type=transaction/date=2023-09-27/hour=7/batch_id=10/part-00000-a9f8364c-705b-4758-a8d9-57b67ff2d236.c000.gz.parquet')

[2023-09-30 18:21:03] {MetricsConfig} WARN: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
df.show(100, False)

+------------------------------------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|object_id                           |sent_dttm          |payload                                                                                                                                                                                                                                                                        |trigger_dttm              |
+------------------------------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
df.printSchema()

root
 |-- object_id: string (nullable = true)
 |-- sent_dttm: timestamp (nullable = true)
 |-- payload: string (nullable = true)
 |-- trigger_dttm: timestamp (nullable = true)

